<a href="https://colab.research.google.com/github/NehaSontakk/BATH-Analytics/blob/main/Update1_Deduplication_of_BATH_tbl_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def compare_contig_names(original_contigs, positive_strand, negative_strand):
    # Create sets of contig names for easier comparison
    original_contig_names = set(original_contigs.keys())
    positive_strand_names = set(positive_strand.keys())
    negative_strand_names = set(negative_strand.keys())

    # Check for contig names present in all dictionaries
    in_all_three = original_contig_names.intersection(positive_strand_names, negative_strand_names)

    # Print results
    print("Contig names in all three dictionaries:")
    print(len(in_all_three))
    for contig_name in in_all_three:
        print(contig_name)

    # Additional information: contig names unique to each dictionary
    unique_to_original = original_contig_names - (positive_strand_names.union(negative_strand_names))
    unique_to_positive = positive_strand_names - (negative_strand_names)
    unique_to_negative = negative_strand_names - (positive_strand_names)

    if unique_to_original:
        print("\nContig names unique to original contigs:")
        for contig_name in unique_to_original:
            print(contig_name)
            print(len(contig_name))

    if unique_to_positive:
        print("\nContig names unique to positive strand (not neg):")
        print(len(unique_to_positive))
        for contig_name in unique_to_positive:
            print(contig_name)

    if unique_to_negative:
        print("\nContig names unique to negative strand (not pos):")
        print(len(unique_to_negative))
        for contig_name in unique_to_negative:
            print(contig_name)


In [2]:
def count_total_entries(contigs):
    total_entries = 0
    for target_name in contigs:
        total_entries += len(contigs[target_name])
    return total_entries

def process_file(filepath, e_value_cutoff=None):
    print("Processing File.")
    contigs = {}
    header_lines = []
    count = 0

    with open(filepath, 'r') as file:
        lines = file.readlines()

    header_lines = [line for line in lines if line.startswith('#')]
    data_lines = [line for line in lines if not line.startswith('#') and line.strip()]

    for line in data_lines:
        parts = line.split()
        if len(parts) < 4:
            continue

        target_name = parts[0]
        ali_from = int(parts[8])
        ali_to = int(parts[9])
        e_value = float(parts[12])

        # Check if the e-value is below the cutoff (if specified) before adding
        if e_value_cutoff is None or e_value <= e_value_cutoff:
            if target_name not in contigs:
                contigs[target_name] = []

            contigs[target_name].append({
                'ali_from': ali_from,
                'ali_to': ali_to,
                'e_value': e_value,
                'full_line': line.strip()
            })

    # Sorting each contig's entries
    for target_name in contigs:
        contigs[target_name] = sorted(contigs[target_name], key=lambda x: (x['ali_from']))

    # Sorting contigs by name
    contigs = dict(sorted(contigs.items()))

    print("Original File Length", count)
    print("Contig Length:", len(contigs))
    print("Contig Value Entry Length:", count_total_entries(contigs))
    return contigs, header_lines


def find_pos_neg_strand(contigs):
    print("Strand Information.")
    positive_strand = {}
    negative_strand = {}

    for target_name in contigs:
        for entry in contigs[target_name]:
            # Assuming positive strand is when 'ali_from' is less than or equal to 'ali_to'
            if entry['ali_from'] <= entry['ali_to']:
                if target_name not in positive_strand:
                    positive_strand[target_name] = []
                positive_strand[target_name].append(entry)
            # Assuming negative strand is when 'ali_from' is greater than 'ali_to'
            else:
                if target_name not in negative_strand:
                    negative_strand[target_name] = []
                negative_strand[target_name].append(entry)

    # Sorting the positive strand by 'ali_from'
    for target_name in positive_strand:
        positive_strand[target_name] = sorted(positive_strand[target_name], key=lambda x: x['ali_from'])

    # Sorting the negative strand by 'ali_to'
    for target_name in negative_strand:
        negative_strand[target_name] = sorted(negative_strand[target_name], key=lambda x: x['ali_to'], reverse=True)

    print('Contigs after Positive, Negative strand division: ', len(positive_strand) + len(negative_strand))
    return positive_strand, negative_strand

'''
def calculate_overlap(start1, end1, start2, end2):
    overlap = max(0, min(end1, end2) - max(start1, start2))
    #print(overlap)
    total_length = min(end1 - start1, end2 - start2)
    #print(total_length)
    return overlap / total_length if total_length > 0 else 0

def calculate_overlap_negative_strand(start1, end1, start2, end2):
    overlap_start = max(end1, end2)
    overlap_end = min(start1, start2)
    overlap = max(0, overlap_end - overlap_start)
    total_length = min(start1 - end1, start2 - end2)

    # Debugging print statements
    #print(f"Comparing intervals: ({end1}, {start1}) and ({end2}, {start2})")
    #print(f"Overlap start: {overlap_start}, Overlap end: {overlap_end}, Overlap: {overlap}, Total length: {total_length}")

    return overlap / total_length if total_length > 0 else 0
'''

def calculate_overlap(start1, end1, start2, end2):
    print("In calculate overlap +ve strand.")
    # Calculate the actual overlapping region
    overlap_start = max(start1,start2)
    overlap_end = min(end1,end2)
    overlap = max(0, (overlap_end - overlap_start) + 1)
    # Calculate the length of each alignment
    length1 = abs(end1 - start1) + 1
    #print(length1)
    length2 = abs(end2 - start2) + 1
    #print(length2)
    # Calculate overlap as a percentage of the length of each alignment
    overlap_percentage1 = (overlap / length1) if length1 > 0 else 0
    #print(overlap_percentage1)
    overlap_percentage2 = (overlap / length2) if length2 > 0 else 0
    #print(overlap_percentage2)
    # Return the larger of the two percentages
    return max(overlap_percentage1, overlap_percentage2)

def calculate_overlap_negative_strand(start1, end1, start2, end2):
    print("In calculate overlap -ve strand.")
    overlap_start = max(end1, end2)
    overlap_end = min(start1, start2)
    overlap = max(0, (overlap_end - overlap_start)+1)
    total_length = min(start1 - end1, start2 - end2)

    # Calculate the length of each alignment
    length1 = abs(end1 - start1) + 1
    print(length1)
    length2 = abs(end2 - start2) + 1
    print(length2)

    # Calculate overlap as a percentage of the length of each alignment
    overlap_percentage1 = (overlap / length1) if length1 > 0 else 0
    #print(overlap_percentage1)
    overlap_percentage2 = (overlap / length2) if length2 > 0 else 0

    return max(overlap_percentage1, overlap_percentage2)


def add_entry_to_dict(dictionary, target_name, entry):
    if target_name not in dictionary:
        dictionary[target_name] = []
    dictionary[target_name].append(entry)


In [3]:
def update_alignments(contig_dict, target_name, i, j, overlap_below_70_old_discard, overlap_below_70_updated_add, update_first_alignment):
    print("Updating alignments")
    align1 = contig_dict[target_name][i]
    align2 = contig_dict[target_name][j]
    updated_alignment = None  # Initialize to None

    if update_first_alignment:
        # Update align1 based on align2's position
        updated_align1 = align1.copy()

        # Adjust the end of align1 if it overlaps with the start of align2
        if align1['ali_to'] >= align2['ali_from']:
            updated_align1['ali_to'] = align2['ali_from'] - 1
            contig_dict[target_name][i] = updated_align1

        updated_alignment = updated_align1


    else:

        # Update align2 based on align1's position
        updated_align2 = align2.copy()
        if align1['ali_to'] > align2['ali_from']:
            updated_align2['ali_from'] = align1['ali_to'] + 1  # Corrected this line
            contig_dict[target_name][j] = updated_align2
        elif align1['ali_from'] < align2['ali_to']:
            updated_align2['ali_to'] = align1['ali_from'] - 1
            contig_dict[target_name][j] = updated_align2
        updated_alignment = updated_align2

    if updated_alignment:
        add_entry_to_dict(overlap_below_70_old_discard, target_name, align1 if update_first_alignment else align2)
        add_entry_to_dict(overlap_below_70_updated_add, target_name, updated_alignment)

    # Optional debug prints
    print("Final entry values:")
    print("Flag: update_first_alignment", update_first_alignment)
    print("Alignment 1: ", contig_dict[target_name][i]['ali_from'], '\t', contig_dict[target_name][i]['ali_to'], '\t', contig_dict[target_name][i]['e_value'])
    print("Alignment 2: ", contig_dict[target_name][j]['ali_from'], '\t', contig_dict[target_name][j]['ali_to'], '\t', contig_dict[target_name][j]['e_value'])

In [4]:
import json

def save_dict_to_json(contig_dict, filename):
    # Convert the dictionary to a JSON string
    json_string = json.dumps(contig_dict, indent=4)

    # Write the JSON string to a file
    with open(filename, 'w') as file:
        file.write(json_string)


In [5]:
def count_alignments_in_file(data):
    # Count the number of alignments for each contig
    contig_counts = {contig: len(alignments) for contig, alignments in data.items()}

    # Calculate the total number of alignments across all contigs
    total_alignments = sum(contig_counts.values())

    return contig_counts, total_alignments



In [6]:
def deduplication_logic2(contig_dict, is_positive):
    print("In deduplication logic 2.")
    overlap_above_70_keep = {}
    overlap_above_70_discard = {}
    overlap_below_70_updated_add = {}
    overlap_below_70_old_discard = {}

    for target_name in contig_dict:
        contig_dict[target_name] = sorted(contig_dict[target_name], key=lambda x: (x['ali_from'], x['e_value']))

    for target_name in contig_dict:
        i = 0
        while i < len(contig_dict[target_name]):
            align1 = contig_dict[target_name][i]
            j = i + 1
            while j < len(contig_dict[target_name]):
                align2 = contig_dict[target_name][j]
                overlap = (calculate_overlap if is_positive else calculate_overlap_negative_strand)(
                    align1['ali_from'], align1['ali_to'], align2['ali_from'], align2['ali_to']
                )
                # Print overlap details for every pair of alignments
                print(f"Comparing alignments: {align1['full_line']} and {align2['full_line']}")
                print(f"Overlap: {overlap*100:.2f}%")

                if overlap > 0.7:
                  if align1['e_value'] > align2['e_value']:
                      print("Here 1")
                      # Keep align2 in contig_dict
                      # Discard align1 from contig_dict
                      add_entry_to_dict(overlap_above_70_discard, target_name, align1)
                      contig_dict[target_name].pop(i)
                      # No need to increment i, as the next element shifts to the current position
                      # Break out of the inner loop since align1 is removed
                      break
                  elif align1['e_value'] <= align2['e_value']:
                      print("Here 2")
                      # Keep align1 in contig_dict
                      # Discard align2 from contig_dict
                      add_entry_to_dict(overlap_above_70_discard, target_name, align2)
                      contig_dict[target_name].pop(j)
                      # Continue with the next comparison
                      # j is incremented in the loop no need for an additional increment here
                      continue
                  elif align1['e_value'] == align2['e_value']:
                      length1 = align1['ali_to'] - align1['ali_from']
                      length2 = align2['ali_to'] - align2['ali_from']
                      if length1 > length2:
                        print("Keeping longer alignment1")
                        add_entry_to_dict(overlap_above_70_discard, target_name, align2)
                        contig_dict[target_name].pop(j)
                        continue
                      else:
                        print("Keeping longer or equal length alignment2")
                        add_entry_to_dict(overlap_above_70_discard, target_name, align1)
                        contig_dict[target_name].pop(i)
                        break
                elif 0.1 < overlap <= 0.7:
                    print("Here 3")
                    print(f"Moderate overlap detected between: {align1['full_line']} and {align2['full_line']}, Overlap: {overlap}")
                    if align1['e_value'] > align2['e_value']:
                      print("\nCase where align2 is best hit, so update align1")
                      update_alignments(contig_dict, target_name, i, j, overlap_below_70_old_discard, overlap_below_70_updated_add, True)  # Update align1
                    else:
                      print("\nCase where align1 is best hit, so update align2")
                      update_alignments(contig_dict, target_name, i, j, overlap_below_70_old_discard, overlap_below_70_updated_add, False)  # Update align2

                j += 1
            i += 1

    print(contig_dict)
    return contig_dict, overlap_above_70_discard



In [7]:
# Function to combine positive and negative strand data into a tbl file
def combine_strands_to_tbl(positive_strand_data, negative_strand_data, output_filepath):
    combined_lines = []
    for contig in positive_strand_data:
        combined_lines.extend([alignment['full_line'] for alignment in positive_strand_data[contig]])
    for contig in negative_strand_data:
        combined_lines.extend([alignment['full_line'] for alignment in negative_strand_data[contig]])

    with open(output_filepath, 'w') as file:
        file.write("# target name\taccession\tquery name\taccession\thmm len\thmm from\thmm to\tseq len\tali from\tali to\tenv from\tenv to\tE-value\tscore\tbias\tshifts\tstops\tpipe\tdescription of target\n")
        for line in combined_lines:
            file.write(line + "\n")

In [8]:
import pandas as pd

def combine_strands_to_csv_and_tbl(positive_strand_data, negative_strand_data, output_base_path):
    # Define the columns for the DataFrame
    columns = ["target name", "accession", "query name", "accession", "hmm len", "hmm from", "hmm to", "seq len", "ali from", "ali to", "env from", "env to", "E-value", "score", "bias", "shifts", "stops", "pipe", "description of target"]

    # Initialize a list to store data rows
    data_rows = []

    # Process positive strand data
    for contig in positive_strand_data:
        for alignment in positive_strand_data[contig]:
            split_line = alignment['full_line'].split('\t')
            if len(split_line) != len(columns):
                print(f"Error: expected {len(columns)} fields, got {len(split_line)} fields in line: {alignment['full_line']}")
                continue
            data_rows.append(alignment['full_line'].split('\t'))

    # Process negative strand data
    for contig in negative_strand_data:
        for alignment in negative_strand_data[contig]:
            split_line = alignment['full_line'].split('\t')
            if len(split_line) != len(columns):
                print(f"Error: expected {len(columns)} fields, got {len(split_line)} fields in line: {alignment['full_line']}")
                continue
            data_rows.append(alignment['full_line'].split('\t'))

    # Create a DataFrame from the data rows
    df = pd.DataFrame(data_rows, columns=columns)

    # Save as TSV
    tsv_output_path = f"{output_base_path}.tbl"
    df.to_csv(tsv_output_path, sep="\t", index=False, header=True)

    # Save as CSV
    csv_output_path = f"{output_base_path}.csv"
    df.to_csv(csv_output_path, sep=",", index=False, header=True)

    return tsv_output_path, csv_output_path


 Analysis

In [9]:
def save_counts_to_file(positive_counts, negative_counts, original_counts, output_file_path):
    with open(output_file_path, 'w') as file:
        file.write("Contig\tPositive Strand\tNegative Strand\tOriginal Contigs\tTotal (Positive + Negative)\n")

        for contig in set(original_counts.keys()).union(positive_counts.keys(), negative_counts.keys()):
            pos_count = positive_counts.get(contig, 0)
            neg_count = negative_counts.get(contig, 0)

            pos_count = len(pos_count) if isinstance(pos_count, list) else pos_count
            neg_count = len(neg_count) if isinstance(neg_count, list) else neg_count
            orig_count = len(original_counts.get(contig, []))

            total_count = pos_count + neg_count
            line = f"{contig}\t{pos_count}\t{neg_count}\t{orig_count}\t{total_count}\n"
            file.write(line)

    return output_file_path

Running a single file

In [10]:
input_filepath = "/content/test_data_k127_1041423.txt"
e_value_cutoff = 0.000001
output_filepath = "/content/xxx.tbl"
output_filepath_analysis = "/content/here.txt"

contigs, header = process_file(input_filepath,e_value_cutoff)
positive_strand, negative_strand = find_pos_neg_strand(contigs)

contig_neg, discard_neg = deduplication_logic2(negative_strand,is_positive=False)
contig_pos, discard_pos = deduplication_logic2(positive_strand,is_positive=True)
print("hello")
#combine_strands_to_tbl(contig_pos,contig_neg,output_filepath)
tsv_path, csv_path = combine_strands_to_csv_and_tbl(contig_pos, contig_neg, "/content/")
save_counts_to_file(contig_pos,contig_neg,contigs,output_filepath_analysis)

Processing File.
Original File Length 0
Contig Length: 1
Contig Value Entry Length: 5
Strand Information.
Contigs after Positive, Negative strand division:  1
In deduplication logic 2.
{}
In deduplication logic 2.
In calculate overlap +ve strand.
Comparing alignments: k127_1041423	-	MF_00548	-	269	16	260	87228	64419	65129	64374	65144	1.3e-23	92.2	7.7	0	0	std	- and k127_1041423	-	MF_01016	-	553	1	545	87228	71334	72950	71334	72959	3.8e-106	364.6	13.9	0	0	std	-
Overlap: 0.00%
In calculate overlap +ve strand.
Comparing alignments: k127_1041423	-	MF_00548	-	269	16	260	87228	64419	65129	64374	65144	1.3e-23	92.2	7.7	0	0	std	- and k127_1041423	-	MF_01015	-	561	14	284	87228	71355	72146	71334	72278	5.3e-20	80.1	3.8	0	0	std	-
Overlap: 0.00%
In calculate overlap +ve strand.
Comparing alignments: k127_1041423	-	MF_00548	-	269	16	260	87228	64419	65129	64374	65144	1.3e-23	92.2	7.7	0	0	std	- and k127_1041423	-	MF_00236	-	78	4	56	87228	78466	78633	78457	78681	2.3e-10	48.8	0.1	0	0	std	-
Overlap: 0.00%
I

'/content/here.txt'

In [22]:
contigs

{}

Calling Functions

In [ ]:
from glob import glob

for i in glob("/content/drive/MyDrive/Lab Work/Parkinsons_Data/Bath_Output/*.tbl"):
  input_filepath = i
  e_value_cutoff = 0.000001
  !mkdir /content/Deduplicated_Bath_Output/
  output_filepath = "/content/Deduplicated_Bath_Output/"+i.split(".")[0].split("/")[-1]+"_deduplicated.tbl"
  output_filepath_analysis = "/content/Deduplicated_Bath_Output/"+i.split(".")[0].split("/")[-1]+"_counts.txt"
  contigs, header = process_file(input_filepath,e_value_cutoff)
  positive_strand, negative_strand = find_pos_neg_strand(contigs)
  contig_neg_0, discard_neg_0 = deduplication_logic2(negative_strand,is_positive=False)
  contig_neg_1, discard_neg_1 = deduplication_logic2(contig_neg_0,is_positive=False)
  contig_neg, discard_neg = deduplication_logic2(contig_neg_1,is_positive=False)
  print("Negative alignments: \n",count_total_entries(contig_neg))
  contig_pos_0, discard_pos_0 = deduplication_logic2(positive_strand,is_positive=True)
  contig_pos_1, discard_pos_1 = deduplication_logic2(contig_pos_0,is_positive=True)
  contig_pos, discard_pos = deduplication_logic2(contig_pos_1,is_positive=True)
  print("Positive alignments: \n",count_total_entries(contig_pos))
  combine_strands_to_tbl(contig_pos,contig_neg,output_filepath)
  save_counts_to_file(contig_pos,contig_neg,contigs,output_filepath_analysis)
  count_alignments_in_file(contig_neg)

Streaming output truncated to the last 5000 lines.
Overlap: 0.00%
In calculate overlap +ve strand.
Comparing alignments: k127_1013475         -          Q35920               -                227        65       223    183529      4541      5035      4499      5047   1.2e-07   38.6   9.4       0      0   std - and k127_1013475         -          P41309               -                602        66       499    183529     11578     12972     11503     13275   3.8e-55  196.0  26.8       0      0   std -
Overlap: 0.00%
In calculate overlap +ve strand.
Comparing alignments: k127_1013475         -          Q35920               -                227        65       223    183529      4541      5035      4499      5047   1.2e-07   38.6   9.4       0      0   std - and k127_1013475         -          P15552               -                637        98       469    183529     11578     12762     11506     13008   6.1e-46  165.8  17.6       0      0   std -
Overlap: 0.00%
In calculate overlap +ve s

In [ ]:




python /xdisk/twheeler/nsontakke/BATH_Prokka_DB/Scripts/deduplication_of_bath_tbl_output.py /xdisk/twheeler/nsontakke/BATH_Prokka_DB/Output_Deduplication/BATH_output_combined.tbl 0.000001 /xdisk/twheeler/nsontakke/BATH_Prokka_DB/Output_Deduplication/Deduplicated_BATH_output_combined.tbl /xdisk/twheeler/nsontakke/BATH_Prokka_DB/Output_Deduplication/Deduplicated_BATH_combined_analysis.txt


In [ ]:




python /xdisk/twheeler/nsontakke/BATH_Prokka_DB/Scripts/deduplication_of_bath_tbl_output.py /xdisk/twheeler/nsontakke/BATH_Prokka_DB/Output_Deduplication/Deduplicated_BATH_output_combined1.tbl 0.000001 /xdisk/twheeler/nsontakke/BATH_Prokka_DB/Output_Deduplication/Deduplicated_BATH_output_combined2.tbl /xdisk/twheeler/nsontakke/BATH_Prokka_DB/Output_Deduplication/Deduplicated_BATH_combined_analysis2.txt
